In [1]:
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.max_open_warning'] = 100
matplotlib.rcParams['lines.linewidth'] = 1.0

In [3]:
from IPython.display import display

def print_full_df(df):
    pd.set_option('display.max_rows', len(df))
    display(df)
    pd.reset_option('display.max_rows')

# Grupos

## Socios activos 2016

In [4]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [5]:
%%time

dtypes = {
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Edad': 'category',
}

perfiles_df = pd.read_csv(
    'valores_grupos_2016.csv', header=0, dtype=dtypes)
# perfiles_df = perfiles_df.fillna('')

CPU times: user 73 ms, sys: 4.57 ms, total: 77.6 ms
Wall time: 76.9 ms


In [6]:
perfiles_df.shape

(50272, 9)

In [7]:
perfiles_df.head()

,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Edad
0,BLAA,OTRO,chapinero,bogota,CIN,universidad,OTRO,OTRO,OTRO
1,BLAA,sra.,OTRO,bogota,CEM,pensionado,OTRO,OTRO,50-64
2,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,OTRO
3,ARM,sr.,OTRO,OTRO,CEM,empleado,profesional,economía,50-64
4,BLAA,sr.,soledad,bogota,CEM,OTRO,OTRO,OTRO,OTRO


# Eliminando valores en blanco

In [8]:
perfiles_df = perfiles_df[['Ciudad', 'Nivel Estudio1', 'Área de Estudio1']]

In [9]:
perfiles_df = perfiles_df[
    (perfiles_df['Ciudad'] != 'OTRO')
]

In [10]:
perfiles_df.shape

(47456, 3)

In [11]:
perfiles_df.head()

,Ciudad,Nivel Estudio1,Área de Estudio1
0,bogota,OTRO,OTRO
1,bogota,OTRO,OTRO
2,bogota,profesional,filosofía y letras
4,bogota,OTRO,OTRO
5,bogota,OTRO,OTRO


# Agrupando

## Todas las ciudades

In [12]:
ciudades_sucursales = {
    'armenia': 'ARM',
    'barranquilla': 'BAR',
    'bogota': 'BLAA',
    'bucaramanga': 'BUC',
    'buenaventura': 'BUE',
    'cali': 'CAL',
    'cartagena': 'CAR',
    'cucuta': 'CUC',
    'florencia': 'FLO',
    'girardot': 'GIR',
    'honda': 'HON',
    'ibague': 'IBA',
    'ipiales': 'IPI',
    'leticia': 'LET',
    'manizales': 'MAN',
    'medellin': 'MED',
    'monteria': 'MON',
    'neiva': 'NEI',
    'pasto': 'PAS',
    'pereira': 'PER',
    'popayan': 'POP',
    'quibdo': 'QUI',
    'riohacha': 'RIO',
    'san andres': 'SAN',
    'santa marta': 'STM',
    'sincelejo': 'SIN',
    'tunja': 'TUN',
    'valledupar': 'VAL',
    'villavicencio': 'VIL',
}

sucursales = {
    'ARM': {'name': 'Armenia', 'latitude': 4.53, 'longitude': -75.68, 'weight': 575},
    'BAR': {'name': 'Barranquilla', 'latitude': 10.963889, 'longitude': -74.796389, 'weight': 328},
    'BLAA': {'name': 'Bogotá', 'latitude': 4.59677, 'longitude': -74.072848, 'weight': 24284},
    'BUC': {'name': 'Bucaramanga', 'latitude': 7.133333, 'longitude': -73.0, 'weight': 511},
    'BUE': {'name': 'Buenaventura', 'latitude': 3.877222, 'longitude': -77.026667, 'weight': 337},
    'CAL': {'name': 'Cali', 'latitude': 3.420556, 'longitude': -76.522222, 'weight': 287},
    'CAR': {'name': 'Cartagena', 'latitude': 10.4, 'longitude': -75.5, 'weight': 758},
    'CUC': {'name': 'Cúcuta', 'latitude': 7.894167, 'longitude': -72.503889, 'weight': 236},
    'FLO': {'name': 'Florencia', 'latitude': 1.614167, 'longitude': -75.611667, 'weight': 611},
    'GIR': {'name': 'Girardot', 'latitude': 4.305, 'longitude': -74.801667, 'weight': 1090},
    'HON': {'name': 'Honda', 'latitude': 5.204167, 'longitude': -74.741667, 'weight': 544},
    'IBA': {'name': 'Ibagué', 'latitude': 4.433333, 'longitude': -75.233333, 'weight': 2339},
    'IPI': {'name': 'Ipiales', 'latitude': 0.830278, 'longitude': -77.644444, 'weight': 1041},
    'LET': {'name': 'Leticia', 'latitude': -4.205278, 'longitude': -69.932778, 'weight': 526},
    'MAN': {'name': 'Manizales', 'latitude': 5.1, 'longitude': -75.55, 'weight': 1159},
    'MED': {'name': 'Medellín', 'latitude': 6.230833, 'longitude': -75.590556, 'weight': 219},
    'MON': {'name': 'Montería', 'latitude': 8.75, 'longitude': -75.883333, 'weight': 209},
    'NEI': {'name': 'Neiva', 'latitude': 2.998611, 'longitude': -75.304444, 'weight': 1978},
    'PAS': {'name': 'Pasto', 'latitude': 1.207778, 'longitude': -77.277222, 'weight': 2089},
    'PER': {'name': 'Pereira', 'latitude': 4.814278, 'longitude': -75.694558, 'weight': 1922},
    'POP': {'name': 'Popayán', 'latitude': 2.454167, 'longitude': -76.609167, 'weight': 2078},
    'QUI': {'name': 'Quibdó', 'latitude': 5.692222, 'longitude': -76.658056, 'weight': 478},
    'RIO': {'name': 'Riohacha', 'latitude': 11.544167, 'longitude': -72.906944, 'weight': 250},
    'SAN': {'name': 'San Andrés', 'latitude': 12.583333, 'longitude': -81.7, 'weight': 728},
    'SIN': {'name': 'Sincelejo', 'latitude': 9.295, 'longitude': -75.396111, 'weight': 438},
    'STM': {'name': 'Santa Marta', 'latitude': 11.241944, 'longitude': -74.205278, 'weight': 438},
    'TUN': {'name': 'Tunja', 'latitude': 5.533333, 'longitude': -73.366667, 'weight': 3165},
    'VAL': {'name': 'Valledupar', 'latitude': 10.483333, 'longitude': -73.25, 'weight': 660},
    'VIL': {'name': 'Villavicencio', 'latitude': 4.15, 'longitude': -73.633333, 'weight': 298},
}

# weights obtenidos de perfiles_df['Sucursal'].value_counts()

### Top 1 nivel de estudios por ciudad

In [13]:
niveles_ciudades = perfiles_df[perfiles_df['Nivel Estudio1'] != 'OTRO'] \
    .groupby(['Ciudad'])['Nivel Estudio1'] \
    .agg(lambda x: x.value_counts().index[0])

In [14]:
top_niveles_ciudades_ls = [
    (x, y)
    for x, y in niveles_ciudades.iteritems()
    if x in ciudades_sucursales
]

In [15]:
top_niveles_ciudades_df = pd.DataFrame(top_niveles_ciudades_ls, columns=['Ciudad', 'Top nivel de estudios'])

In [16]:
print_full_df(top_niveles_ciudades_df)

,Ciudad,Top nivel de estudios
0,armenia,profesional
1,barranquilla,universitario 2 nivel
2,bogota,universitario 1 nivel
3,bucaramanga,profesional
4,buenaventura,básica primaria
5,cali,profesional
6,cartagena,universitario 2 nivel
7,cucuta,universitario 1 nivel
8,florencia,básica primaria
9,girardot,secundaria


In [17]:
niveles_g = nx.Graph()

for i, n in top_niveles_ciudades_df.iterrows():
    nivel = n['Top nivel de estudios']
    ciudad = n['Ciudad']
    sucursal = ciudades_sucursales[ciudad]
    name = sucursales[sucursal]['name']
    lat = sucursales[sucursal]['latitude']
    long = sucursales[sucursal]['longitude']
    weight = sucursales[sucursal]['weight']
    
    niveles_g.add_node(
        sucursal,
        name=name,
        level=nivel,
        latitude=lat,
        longitude=long,
        weight=weight,
    )

In [18]:
top_niveles_ciudades_df['Top nivel de estudios'].unique()

array(['profesional', 'universitario 2 nivel', 'universitario 1 nivel',
       'básica primaria', 'secundaria'], dtype=object)

In [19]:
nx.write_gexf(niveles_g, 'niveles_estudio_ciudades.gexf')

### Top 1 área de estudios por ciudad

In [20]:
areas_ciudades = perfiles_df[perfiles_df['Área de Estudio1'] != 'OTRO'] \
    .groupby(['Ciudad'])['Área de Estudio1'] \
    .agg(lambda x: x.value_counts().index[0])

In [21]:
top_areas_ciudades_ls = [
    (x, y)
    for x, y in areas_ciudades.iteritems()
    if x in ciudades_sucursales
]

In [22]:
top_areas_ciudades_df = pd.DataFrame(top_areas_ciudades_ls, columns=['Ciudad', 'Top área de estudios'])

In [23]:
print_full_df(top_areas_ciudades_df)

,Ciudad,Top área de estudios
0,armenia,literatura
1,barranquilla,sociología
2,bogota,derecho
3,bucaramanga,historia
4,buenaventura,adm. de empresas
5,cali,historia
6,cartagena,derecho
7,cucuta,derecho
8,florencia,derecho
9,girardot,escolar


In [24]:
areas_g = nx.Graph()

for i, n in top_areas_ciudades_df.iterrows():
    area = n['Top área de estudios']
    ciudad = n['Ciudad']
    sucursal = ciudades_sucursales[ciudad]
    name = sucursales[sucursal]['name']
    lat = sucursales[sucursal]['latitude']
    long = sucursales[sucursal]['longitude']
    weight = sucursales[sucursal]['weight']
    
    areas_g.add_node(
        sucursal,
        name=name,
        field=area,
        latitude=lat,
        longitude=long,
        weight=weight,
    )

In [25]:
top_areas_ciudades_df['Top área de estudios'].unique()

array(['literatura', 'sociología', 'derecho', 'historia',
       'adm. de empresas', 'escolar', 'contaduría', 'ciencias sociales',
       'idiomas'], dtype=object)

In [26]:
nx.write_gexf(areas_g, 'areas_estudio_ciudades.gexf')